# import libraries and dataset
the dataset used here are [crop_recommendation.csv](https://www.kaggle.com/datasets/atharvaingle/crop-recommendation-dataset)
and the modefied dataset of [Urban_Areas.csv](https://www.kaggle.com/datasets/thedevastator/urban-areas-dataset)

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

# Load datasets
urban_df = pd.read_csv('Cleaned_Urban_Areas.csv')  # Cleaned Urban Areas dataset
crop_df = pd.read_csv('crop_recommendation.csv')  # Crop Recommendation dataset

# Display the first few rows to verify data
print("Urban Data:\n", urban_df.head())
print("\nCrop Data:\n", crop_df.head())

Urban Data:
    Zone_ID   Latitude  Longitude      Area    N   P   K  Temperature  \
0     2001  33.525411 -91.438467   5064930  101  73  41    34.214295   
1     2002  35.612144 -91.262068  11738347  142  28  55    21.409738   
2     2003  36.063002 -90.965963   9595023   64  56  66    34.925936   
3     2004  33.622119 -91.793763  25182217  121  68  77    31.911024   
4     2005  30.129085 -91.835324  12639297  110  60  62    27.219190   

    Humidity    Rainfall        pH  
0  58.613983  288.598308  7.979513  
1  45.407538  260.468752  6.275308  
2  52.579948  397.327744  5.527830  
3  53.598255  254.397465  6.415805  
4  76.033393  298.008041  5.705615  

Crop Data:
     N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.86

# Define AVL tree class

In [21]:
class Node:
    def __init__(self, key, zone_data):
        self.key = key
        self.zone_data = zone_data
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def get_height(self, root):
        if not root:
            return 0
        return root.height
    
    def get_balance(self, root):
        if not root:
            return 0
        return self.get_height(root.left) - self.get_height(root.right)
    
    def right_rotate(self, z):
        y = z.left
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y
    
    def left_rotate(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y
    
    def insert(self, root, key, zone_data):
        if not root:
            return Node(key, zone_data)
        if key < root.key:
            root.left = self.insert(root.left, key, zone_data)
        elif key > root.key:
            root.right = self.insert(root.right, key, zone_data)
        else:
            return root
        
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        balance = self.get_balance(root)
        
        # Left Heavy
        if balance > 1 and key < root.left.key:
            return self.right_rotate(root)
        # Right Heavy
        if balance < -1 and key > root.right.key:
            return self.left_rotate(root)
        # Left-Right Case
        if balance > 1 and key > root.left.key:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)
        # Right-Left Case
        if balance < -1 and key < root.right.key:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)
        
        return root
    
    def find_zone(self, root, key):
        if not root:
            return None
        if key == root.key:
            return root
        elif key < root.key:
            return self.find_zone(root.left, key)
        else:
            return self.find_zone(root.right, key)

# Insert data into the AVL tree

In [23]:
# Initialize AVL Tree
zones = AVLTree()
root = None

# Map Urban dataset to AVL Tree
for index, row in urban_df.iterrows():
    zone_data = {
        'latitude': row['Latitude'],
        'longitude': row['Longitude'],
        'area': row['Area'],
        'N': row['N'],
        'P': row['P'],
        'K': row['K'],
        'temperature': row['Temperature'],
        'humidity': row['Humidity'],
        'rainfall': row['Rainfall'],
        'pH': row['pH']
    }
    root = zones.insert(root, row['Zone_ID'], zone_data)

print("AVL Tree Loaded with Field Zones ✅")

AVL Tree Loaded with Field Zones ✅


# Load Crop Recommendation Dataset into Multi-Dimensional Array

In [25]:
# Load Crop Recommendation into Multi-Dimensional Array
crop_data = crop_df[['label', 'N', 'P', 'K', 'temperature', 'humidity', 'ph']].values

# Show example crop data
print("\nExample Crop Data:")
for row in crop_data[:5]:
    print(row)


Example Crop Data:
['rice' 90 42 43 20.87974371 82.00274423 6.502985292000001]
['rice' 85 58 41 21.77046169 80.31964408 7.038096361]
['rice' 60 55 44 23.00445915 82.3207629 7.840207144]
['rice' 74 35 40 26.49109635 80.15836264 6.980400905]
['rice' 78 42 42 20.13017482 81.60487287 7.628472891]


# Define get_health() function

In [27]:
def get_health(crop, zone_id):
    node = zones.find_zone(root, zone_id)
    if not node:
        return f"Zone {zone_id} not found."
    
    zone = node.zone_data

    # Get ideal crop values
    crop_ideal = None
    for row in crop_data:
        if row[0].lower() == crop.lower():
            crop_ideal = row
            break

    if crop_ideal is None:
        return f"{crop} data not found in crop recommendation dataset."

    suggestions = []

    # NPK Suggestions
    if not (crop_ideal[1] - 10 <= zone['N'] <= crop_ideal[1] + 10):
        suggestions.append(f"Nitrogen (N) level mismatch (Expected: {crop_ideal[1]}±10, Found: {zone['N']}). Suggestion: Add nitrogen-rich fertilizer like urea or compost.")
    if not (crop_ideal[2] - 10 <= zone['P'] <= crop_ideal[2] + 10):
        suggestions.append(f"Phosphorus (P) level mismatch (Expected: {crop_ideal[2]}±10, Found: {zone['P']}). Suggestion: Add phosphorus-rich sources like bone meal or rock phosphate.")
    if not (crop_ideal[3] - 10 <= zone['K'] <= crop_ideal[3] + 10):
        suggestions.append(f"Potassium (K) level mismatch (Expected: {crop_ideal[3]}±10, Found: {zone['K']}). Suggestion: Use potash or wood ash to boost potassium.")

    # Temperature Suggestion
    if not (crop_ideal[4] - 20 <= zone['temperature'] <= crop_ideal[4] + 20):
        suggestions.append(f"Temperature mismatch (Expected: {crop_ideal[4]}±20°C, Found: {zone['temperature']}°C). Suggestion: Use shade nets or greenhouses to control temperature.")

    # Humidity Suggestion
    if not (crop_ideal[5] - 26 <= zone['humidity'] <= crop_ideal[5] + 26):
        suggestions.append(f"Humidity mismatch (Expected: {crop_ideal[5]}±26%, Found: {zone['humidity']}%). Suggestion: Use misting systems or dehumidifiers based on requirement.")

    # pH Suggestion
    if not (crop_ideal[6] - 5 <= zone['pH'] <= crop_ideal[6] + 5):
        if zone['pH'] < crop_ideal[6]:
            suggestions.append(f"Soil pH is too low (Expected: {crop_ideal[6]}±5, Found: {zone['pH']}). Suggestion: Add lime to increase pH.")
        else:
            suggestions.append(f"Soil pH is too high (Expected: {crop_ideal[6]}±5, Found: {zone['pH']}). Suggestion: Add sulfur or organic matter to reduce pH.")

    if suggestions:
        result = f"Zone {zone_id} is NOT suitable for growing {crop}.\nSuggestions:\n- " + "\n- ".join(suggestions)
    else:
        result = f"Zone {zone_id} is SUITABLE for growing {crop} ✅"

    return result


# Train AI Model

In [29]:
features = crop_df[['N', 'P', 'K', 'temperature', 'humidity', 'ph']]
labels = crop_df['label']

le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

# AI crop recommendation

In [31]:
def get_current_season(month):
    if month in [6, 7, 8, 9]:
        return 'Kharif'
    elif month in [10, 11, 12, 1]:
        return 'Rabi'
    elif month in [2, 3, 4, 5]:
        return 'Zaid'
    else:
        return 'Unknown'

season_crop_map = {
    'Kharif': ['rice', 'maize', 'cotton', 'soybean'],
    'Rabi': ['wheat', 'barley', 'gram', 'mustard'],
    'Zaid': ['melon', 'cucumber', 'bitter gourd', 'ridge gourd']
}

def recommend_by_season(month=None):
    if month is None:
        month = datetime.now().month
    season = get_current_season(month)
    crops = season_crop_map.get(season, [])
    return f"🌾 Season: {season}\n✅ Recommended Crops: {', '.join(crops)}"

def recommend_by_zone_data(zone_data):
    input_data = [[
        zone_data['N'],
        zone_data['P'],
        zone_data['K'],
        zone_data['temperature'],
        zone_data['humidity'],
        zone_data['pH']
    ]]
    predicted_label = rf_model.predict(input_data)[0]
    crop_name = le.inverse_transform([predicted_label])[0]
    return f"📍 Based on zone data, AI recommends: {crop_name}"

# Testing the code

In [33]:
# Test Cases
print(get_health('rice', 2003))   # Example for rice
print(get_health('wheat', 2001))  # Example for wheat
print(get_health('maize', 2001))  # Example for maize

Zone 2003 is NOT suitable for growing rice.
Suggestions:
- Nitrogen (N) level mismatch (Expected: 90±10, Found: 64.0). Suggestion: Add nitrogen-rich fertilizer like urea or compost.
- Phosphorus (P) level mismatch (Expected: 42±10, Found: 56.0). Suggestion: Add phosphorus-rich sources like bone meal or rock phosphate.
- Potassium (K) level mismatch (Expected: 43±10, Found: 66.0). Suggestion: Use potash or wood ash to boost potassium.
- Humidity mismatch (Expected: 82.00274423±26%, Found: 52.57994848824126%). Suggestion: Use misting systems or dehumidifiers based on requirement.
wheat data not found in crop recommendation dataset.
Zone 2001 is NOT suitable for growing maize.
Suggestions:
- Nitrogen (N) level mismatch (Expected: 71±10, Found: 101.0). Suggestion: Add nitrogen-rich fertilizer like urea or compost.
- Phosphorus (P) level mismatch (Expected: 54±10, Found: 73.0). Suggestion: Add phosphorus-rich sources like bone meal or rock phosphate.
- Potassium (K) level mismatch (Expected

# AI example usage

In [41]:
# Example 1: Crop recommendation for current month
print(recommend_by_season())  # uses system date

# Example 2: Crop recommendation using sample zone data
sample_zone = {
    'N': 73,
    'P': 45,
    'K': 39,
    'temperature': 26,
    'humidity': 44,
    'pH': 6.5
}
print(recommend_by_zone_data(sample_zone))

🌾 Season: Zaid
✅ Recommended Crops: melon, cucumber, bitter gourd, ridge gourd
📍 Based on zone data, AI recommends: coffee


C:\Users\umasi\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
